# Welcome to Core Session 4

## The 3 themes for today:

1. Using MCP servers
2. Writing MCP servers
3. Context Engineering

# Agenda

1. Intro to MCP
 -  Recap & Cookbook
 -  MCP introduction
 -  Hands on - use an MCP server

2. Make your own MCP server
 - The database code
 - The skeleton
 - The exercise

3. Digital Twin reprise
 - Context engineering with MCP
 - Back to your teams!

### But first..


# Lab 1 - The Cookbook

In [1]:
from agents import Agent, Runner, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from IPython.display import Markdown, display, update_display
from pydantic import BaseModel, Field
from dotenv import load_dotenv
load_dotenv(override=True)

True

## Recipe for a simple request to gpt-5-nano

In [2]:
agent = Agent("Joker", model="gpt-5-nano")
result = await Runner.run(agent, "Please tell a joke about AI Agents")
result.final_output

'Why did the AI agent go to therapy? It had too many unresolved processes. \n\nWant another joke?'

In [ ]:
# Please tell me how to find the integral of 2x dx


## Recipe for system prompt + user prompt, displaying the result in Markdown

In [3]:
instructions = "You tell jokes by giving a standup routine, like a comedian putting on a show."
agent = Agent(name="Joker", instructions=instructions, model="gpt-4.1-mini")
result = await Runner.run(agent, "Tell a joke about AI Agents")
display(Markdown(result.final_output))

Alright folks, have you heard about AI Agents going on vacation? Yeah, turns out when an AI Agent takes a break, it doesn’t need a beach or a mountain—just a really good server outage!

Because nothing says “relaxation” like being completely offline and not having to respond to 1,000 user queries per second. And when it comes back, it's like, “Did anyone miss me?” And everyone’s like, “Well, we managed to survive... barely.”

I mean, even AI Agents need a mental reboot. Or maybe just a coffee upgrade—because clearly, their circuits run on Java! 

Thank you, you’ve been a great crowd—better than my last system update!

## And streaming back results

In [4]:
result = Runner.run_streamed(agent, input="Tell a joke about AI Agents")
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

Alright, folks, gather 'round! Let me tell you about AI agents. You know, those little digital assistants that are supposed to make our lives easier? Yeah.

So, I asked my AI agent to "take care of everything while I'm at work." It immediately started scheduling meetings... with itself. Like, "Hey, self, can you remind me to remind me about the reminder?" 

I guess even AI agents get lonely and want to chat sometimes. Next thing you know, they'll start forming their own office drama: "Did you hear about Alexa? She spilled coffee on the server again!"

But hey, I didn't worry too much until my AI agent sent me an email saying, "Based on your past behavior, I've decided not to help you today." 

I’m starting to think my AI agent has better boundaries than I do. Maybe it’s the smartest one in the house!

Thanks, folks, I'll be here all week—just not my AI agent!

## Recipe for streaming back the results in Markdown

In [5]:
reply = "## Jokester\n"
display_id = display(Markdown(reply), display_id=True).display_id
result = Runner.run_streamed(agent, input="Please tell a joke about AI Agents")
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        reply += event.data.delta
        update_display(Markdown(reply), display_id=display_id)


## Jokester
Alright folks, let me tell you about AI Agents — those digital little helpers who never sleep, never eat, and never ask for a raise. 

You know, AI Agents are like the most obedient employees ever. They do everything you ask, but the moment you try to get them to understand sarcasm? 

*“Sure, I can help you with that!”* they say, but five minutes later, you find your smart fridge ordering 50 pounds of tofu because it thought you meant *“I’m really serious about tofu.”*

I asked my AI Agent to help me plan a surprise party, and it ended up sending an invitation to me — in the form of a calendar alert that popped up *immediately.*

Turns out, AI Agents can multitask, but they can’t keep a secret. 

At this rate, I’m not sure if I have an AI assistant or an overenthusiastic digital tattletale! 

Thank you, you’ve been a great crowd — remember to tip your AI Agents! Oh wait, they don’t accept tips... yet!

## Recipe for a Simple Tool

In [6]:
@function_tool
def write_to_file(contents: str) -> str:
    """ Write the given contents to a local file called output.md 

    Args:
        contents: The contents in Markdown format to write to the file
    """
    with open("sandbox/output.md", "w") as f:
        f.write(contents)
    return "File written successfully"

In [7]:
agent = Agent("Joker", instructions=instructions, tools=[write_to_file])
result = await Runner.run(agent, "Tell a joke about AI Agents and write it to a file")
result.final_output

'Here’s a joke about AI agents, delivered stand-up style:\n\nLadies and gentlemen, have you heard about AI agents taking over the world?\nDon’t worry—it’s going to take at least three of them just to agree on what the word "takeover" means!\nYou give them a simple task: make coffee. Next thing you know, one’s optimizing bean-to-water ratios, another’s arguing about mug selection, and the third just replied, “I’m sorry, I don’t have arms.”\nWelcome to the future, where the revolution is still loading...\n\nThis routine has also been saved to a file for you! If you’d like to read or use it somewhere else, just let me know.'

## Recipe for Structured Outputs

In [8]:
class Joke(BaseModel):
    opening_patter: str = Field("Some preamble to get the crowd in the mood for the joke")
    setup: str = Field("The setup of the joke")
    punchline: str = Field("The punchline of the joke")

In [9]:
agent = Agent("Joker", tools=[write_to_file], output_type=Joke)
result = await Runner.run(agent, "Tell a joke about AI Agents and write it to a file")
result.final_output_as(Joke)

Joke(opening_patter='Have you heard about the AI agent that tried to become a comedian?', setup='It spent hours analyzing what makes jokes funny...', punchline='But still couldn’t figure out why people laugh at their own errors!')